In [ ]:
!pip install mlxtend==0.20.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 8.4 MB/s 
  Attempting uninstall: mlxtend
    Found existing installation: mlxtend 0.14.0
    Uninstalling mlxtend-0.14.0:
      Successfully uninstalled mlxtend-0.14.0


資料來源：https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/overview 

In [ ]:
!git clone https://github.com/Wang-Jian-An/Feature_Selection.git

Cloning into 'Feature_Selection'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.


# Import packages

In [ ]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer

from xgboost import XGBRegressor

import warnings
warnings.filterwarnings("ignore")

In [ ]:
raw_data = pd.read_csv("Feature_Selection//train.csv")
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [ ]:
# 把遺失值過多的欄位排除掉
raw_data = raw_data.dropna(axis = 1, thresh = int(round(raw_data.shape[0]*0.5)) )

In [ ]:
# Define Features and Target
PK = "Id"
target = "SalePrice"
numerical_features = [i for i in raw_data.columns if i not in [PK, target] and raw_data[i].dtype != "object"]
classical_features = [i for i in raw_data.columns if i not in [PK, target] and raw_data[i].dtype == "object"]

In [ ]:
# 切割成訓練、驗證與測試資料
xtrain, xtest, ytrain, ytest = train_test_split(raw_data[numerical_features+classical_features], raw_data[target], test_size = 0.2)

In [ ]:
print(xtrain.shape, xtest.shape, ytrain.shape, ytest.shape)

(1168, 75) (292, 75) (1168,) (292,)


In [ ]:
def generate_one_hot_encoding_features(one_train_Series,
                                       one_test_Series):
    
    # 產生 One-Hot Encoding Object
    oneHotEncoding = OneHotEncoder(handle_unknown = "ignore")

    # 用訓練資料配適 One-Hot Encoding
    oneHotEncoding = oneHotEncoding.fit(one_train_Series.values.reshape((-1, 1)))

    # 產生 One-Hot Encoding 的資料型態
    oneHot_train_data = pd.DataFrame(oneHotEncoding.transform(one_train_Series.values.reshape((-1, 1))).toarray(), columns = oneHotEncoding.categories_[0].tolist() )
    oneHot_test_data = pd.DataFrame(oneHotEncoding.transform(one_test_Series.values.reshape((-1, 1))).toarray(), columns = oneHotEncoding.categories_[0].tolist() )

    return oneHot_train_data, oneHot_test_data

In [ ]:
# 把類別資料轉成 One-Hot Encoding
OneHotEncoding_data = [generate_one_hot_encoding_features(one_train_Series = xtrain[one_column], one_test_Series = xtest[one_column]) for one_column in classical_features]

# 建立 One-Hot Encoding 後的訓練資料
preprocessed_xtrain = pd.concat([xtrain.reset_index(drop = True)] + [
    data[0] for data in OneHotEncoding_data
], axis = 1).drop(columns = classical_features)

# 建立 One-Hot Encoding 後的測試資料
preprocessed_xtest = pd.concat([xtest.reset_index(drop = True)]+[
    data[1] for data in OneHotEncoding_data
], axis = 1).drop(columns = classical_features)

In [ ]:
print(preprocessed_xtrain.shape, preprocessed_xtest.shape)

(1168, 281) (292, 281)


In [ ]:
# Imputation
KNNimputation = KNNImputer(weights = "distance")

# preprocessed_xtrain = KNNimputation.fit_transform(preprocessed_xtrain) # 輸出格式為 Array
# preprocessed_xtest = KNNimputation.transform(preprocessed_xtest) # 輸出格式為 Array

preprocessed_xtrain = pd.DataFrame(KNNimputation.fit_transform(preprocessed_xtrain), columns = preprocessed_xtrain.columns.tolist())
preprocessed_xtest = pd.DataFrame(KNNimputation.fit_transform(preprocessed_xtest), columns = preprocessed_xtest.columns.tolist())

In [ ]:
preprocessed_xtrain

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,ConLw,New,Oth,WD,Abnorml,AdjLand,Alloca,Family,Normal,Partial
0,30.0,30.0,5232.0,5.0,5.0,1925.0,2004.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,120.0,37.0,3316.0,8.0,5.0,2005.0,2005.0,0.0,1039.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,72.0,10152.0,5.0,5.0,1956.0,1956.0,120.0,586.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,50.0,77.0,9786.0,6.0,7.0,1962.0,1981.0,0.0,600.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,20.0,88.0,11394.0,9.0,2.0,2010.0,2010.0,350.0,1445.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1163,60.0,80.0,9200.0,8.0,5.0,1998.0,1998.0,473.0,986.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1164,60.0,75.0,9803.0,7.0,5.0,2005.0,2005.0,0.0,400.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1165,80.0,75.0,9750.0,5.0,5.0,1967.0,1967.0,0.0,400.0,480.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1166,90.0,67.0,8777.0,5.0,7.0,1900.0,2003.0,0.0,1084.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
preprocessed_xtest.shape

(292, 281)

# ANOVA

In [ ]:
from scipy.stats import f_oneway

In [ ]:
one_class_column = classical_features[0]

In [ ]:
one_class_column

'MSZoning'

In [ ]:
# 計算有幾個組別
unique_class = raw_data[one_class_column].unique()

In [ ]:
unique_class

array(['RL', 'RM', 'C (all)', 'FV', 'RH'], dtype=object)

In [ ]:
# 拆出五組連續變數
target1 = raw_data[raw_data[one_class_column] == unique_class[0]][target]
target2 = raw_data[raw_data[one_class_column] == unique_class[1]][target]
target3 = raw_data[raw_data[one_class_column] == unique_class[2]][target]
target4 = raw_data[raw_data[one_class_column] == unique_class[3]][target]
target5 = raw_data[raw_data[one_class_column] == unique_class[4]][target]

In [ ]:
target_one_class = (target1, target2, target3, target4, target5)

In [ ]:
f_oneway(target1, target2, target3, target4, target5)

F_onewayResult(statistic=43.84028167245718, pvalue=8.817633866272648e-35)

In [ ]:
f_oneway(*target_one_class)

F_onewayResult(statistic=43.84028167245718, pvalue=8.817633866272648e-35)

In [ ]:
# 判斷某個變數在類別變數之間是否有差異

def identify_difference_from_anova(data, column_name, target_name):
    
    # ANOVA
    f_statistics, f_pvalue = f_oneway(*tuple([data[data[column_name] == one_class][target_name] for one_class in data[column_name].unique()]) )
    
    if f_pvalue < 0.05:
        return column_name
    
significant_column = [identify_difference_from_anova(data = raw_data, column_name = one_column_name, target_name = target) for one_column_name in classical_features]

while None in significant_column:
    significant_column.remove(None)

In [ ]:
significant_column

['MSZoning',
 'LotShape',
 'LandContour',
 'LotConfig',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'KitchenQual',
 'Functional',
 'PavedDrive',
 'SaleType',
 'SaleCondition']

In [ ]:
raw_data.corr()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
Id,1.000000,0.011156,-0.010601,-0.033226,-0.028365,0.012609,-0.012713,-0.021998,-0.050298,-0.005024,...,-0.029643,-0.000477,0.002889,-0.046635,0.001330,0.057044,-0.006242,0.021172,0.000712,-0.021917
MSSubClass,0.011156,1.000000,-0.386347,-0.139781,0.032628,-0.059316,0.027850,0.040581,0.022936,-0.069836,...,-0.012579,-0.006100,-0.012037,-0.043825,-0.026030,0.008283,-0.007683,-0.013585,-0.021407,-0.084284
LotFrontage,-0.010601,-0.386347,1.000000,0.426095,0.251646,-0.059213,0.123349,0.088866,0.193458,0.233633,...,0.088521,0.151972,0.010700,0.070029,0.041383,0.206167,0.003368,0.011200,0.007450,0.351799
LotArea,-0.033226,-0.139781,0.426095,1.000000,0.105806,-0.005636,0.014228,0.013788,0.104160,0.214103,...,0.171698,0.084774,-0.018340,0.020423,0.043160,0.077672,0.038068,0.001205,-0.014261,0.263843
OverallQual,-0.028365,0.032628,0.251646,0.105806,1.000000,-0.091932,0.572323,0.550684,0.411876,0.239666,...,0.238923,0.308819,-0.113937,0.030371,0.064886,0.065166,-0.031406,0.070815,-0.027347,0.790982
OverallCond,0.012609,-0.059316,-0.059213,-0.005636,-0.091932,1.000000,-0.375983,0.073741,-0.128101,-0.046231,...,-0.003334,-0.032589,0.070356,0.025504,0.054811,-0.001985,0.068777,-0.003511,0.043950,-0.077856
YearBuilt,-0.012713,0.027850,0.123349,0.014228,0.572323,-0.375983,1.000000,0.592855,0.315707,0.249503,...,0.224880,0.188686,-0.387268,0.031355,-0.050364,0.004950,-0.034383,0.012398,-0.013618,0.522897
YearRemodAdd,-0.021998,0.040581,0.088866,0.013788,0.550684,0.073741,0.592855,1.000000,0.179618,0.128451,...,0.205726,0.226298,-0.193919,0.045286,-0.038740,0.005829,-0.010286,0.021490,0.035743,0.507101
MasVnrArea,-0.050298,0.022936,0.193458,0.104160,0.411876,-0.128101,0.315707,0.179618,1.000000,0.264736,...,0.159718,0.125703,-0.110204,0.018796,0.061466,0.011723,-0.029815,-0.005965,-0.008201,0.477493
BsmtFinSF1,-0.005024,-0.069836,0.233633,0.214103,0.239666,-0.046231,0.249503,0.128451,0.264736,1.000000,...,0.204306,0.111761,-0.102303,0.026451,0.062021,0.140491,0.003571,-0.015727,0.014359,0.386420


# Exhaustive Feature Selection
評估指標參考連結：https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter 

In [ ]:
from mlxtend.feature_selection import ExhaustiveFeatureSelector

In [ ]:
model = XGBRegressor()

# 建立特徵挑選物件
efs = ExhaustiveFeatureSelector(model,
                 min_features = 10,
                 max_features = 11,
                 scoring = "neg_mean_squared_error",
                 cv = 0)

# 開始執行特徵挑選
efs.fit(preprocessed_xtrain, ytrain)

串流輸出內容已截斷至最後 5000 行。
[08:23:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:23:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:23:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:23:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:23:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:23:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:23:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:23:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now depr

KeyboardInterrupt: ignored

In [ ]:
# 輸出最好的特徵組合
efs.best_feature_names_

# Sequential Forward Selection

程式碼參考連結：http://rasbt.github.io/mlxtend/user_guide/feature_selection/SequentialFeatureSelector/#overview    
評估指標參考連結：https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter 

In [ ]:
# from sklearn.feature_selection import SequentialFeatureSelector
from mlxtend.feature_selection import SequentialFeatureSelector

In [ ]:
model = XGBRegressor()

# 建立特徵挑選物件
sfs = SequentialFeatureSelector(model, 
                 k_features = 10,
                 forward = True,
                 floating = False,
                 cv = 0)

# 開始執行特徵挑選
sfs.fit(preprocessed_xtrain, ytrain)                          

SequentialFeatureSelector(cv=0,
                          estimator=XGBRegressor(base_score=None, booster=None,
                                                 colsample_bylevel=None,
                                                 colsample_bynode=None,
                                                 colsample_bytree=None,
                                                 gamma=None, gpu_id=None,
                                                 importance_type='gain',
                                                 interaction_constraints=None,
                                                 learning_rate=None,
                                                 max_delta_step=None,
                                                 max_depth=None,
                                                 min_child_weight=None,
                                                 missing=nan,
                                                 monotone_constraints=None,
                                                 n_estimators=100, n_jobs=None,
                                                 num_parallel_tree=None,
                                                 random_state=None,
                                                 reg_alpha=None,
                                                 reg_lambda=None,
                                                 scale_pos_weight=None,
                                                 subsample=None,
                                                 tree_method=None,
                                                 validate_parameters=None,
                                                 verbosity=None),
                          k_features=30, scoring='neg_mean_squared_error')

In [ ]:
# 輸出每一輪特徵挑選狀況
sfs.subsets_

{1: {'feature_idx': (15,),
  'cv_scores': array([-9.31386715e+08]),
  'avg_score': -931386715.4045746,
  'feature_names': ('GrLivArea',)},
 2: {'feature_idx': (15, 26),
  'cv_scores': array([-1.50208528e+08]),
  'avg_score': -150208527.6395413,
  'feature_names': ('GrLivArea', 'GarageArea')},
 3: {'feature_idx': (11, 15, 26),
  'cv_scores': array([-56827549.84365914]),
  'avg_score': -56827549.84365914,
  'feature_names': ('TotalBsmtSF', 'GrLivArea', 'GarageArea')},
 4: {'feature_idx': (10, 11, 15, 26),
  'cv_scores': array([-21078696.01360529]),
  'avg_score': -21078696.01360529,
  'feature_names': ('BsmtUnfSF', 'TotalBsmtSF', 'GrLivArea', 'GarageArea')},
 5: {'feature_idx': (6, 10, 11, 15, 26),
  'cv_scores': array([-11212930.26839508]),
  'avg_score': -11212930.268395085,
  'feature_names': ('YearRemodAdd',
   'BsmtUnfSF',
   'TotalBsmtSF',
   'GrLivArea',
   'GarageArea')},
 6: {'feature_idx': (2, 6, 10, 11, 15, 26),
  'cv_scores': array([-7016567.52080409]),
  'avg_score': -701656

In [ ]:
# 輸出被選入的特徵
sfs.k_feature_names_

('LotFrontage',
 'LotArea',
 'OverallQual',
 'YearRemodAdd',
 'MasVnrArea',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'GrLivArea',
 'GarageArea',
 'OpenPorchSF',
 'MoSold',
 'YrSold',
 'IR1',
 'AllPub',
 'NoSeWa',
 'FR3',
 'BrDale',
 'MeadowV',
 'NPkVill',
 'RRNe',
 'RRNn',
 'Artery',
 'PosA',
 'RRNn',
 'Twnhs',
 'TwnhsE',
 'Slab',
 'Fa',
 'Fin',
 'Gd')

# Sequential Backward Selection

In [ ]:
# from sklearn.feature_selection import SequentialFeatureSelector
from mlxtend.feature_selection import SequentialFeatureSelector

In [ ]:
model = XGBRegressor()

# 建立特徵挑選物件
sbs = SequentialFeatureSelector(model,
                 k_features = 200,
                 forward = False,
                 floating = False,
                 cv = 0)

# 開始執行特徵挑選
sbs.fit(preprocessed_xtrain, ytrain)

串流輸出內容已截斷至最後 5000 行。
[04:52:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:52:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:52:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:52:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:52:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:52:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:52:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:52:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now depr

In [ ]:
# 輸出特徵挑選過程


In [ ]:
# 輸出被選入的特徵


# Sequential Floating Forward Selection

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector

In [ ]:
model = XGBRegressor()

# 建立特徵挑選物件
sffs = SequentialFeatureSelector(estimator = model,
                  k_features = 50,
                  scoring = "neg_mean_squared_error",
                  cv = 0,
                  floating = True,
                  forward = True)

# 開始執行特徵挑選
sffs.fit(preprocessed_xtrain, ytrain)

In [ ]:
# 輸出特徵挑選過程
sffs.subsets_

In [ ]:
# 輸出被選入的特徵
sffs.k_feature_names_

# Sequential Floating Backward Selection

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector

In [ ]:
model = XGBRegressor()

# 建立特徵挑選物件
sfbs = SequentialFeatureSelector(estimator = model,
                 k_features = 50,
                 floating = True,
                 cv = 0,
                 forward = False)

# 開始執行特徵挑選
sfbs.fit(preprocessed_xtrain, ytrain)

In [ ]:
# 輸出被挑選到的特徵
sfbs.k_feature_names_

# Recursive Feature Elimination

In [ ]:
from sklearn.feature_selection import RFE

In [ ]:
model = XGBRegressor()

# 建立 RFE 物件
rfe = RFE(estimator = model, 
      n_features_to_select = 200,
      step = 5)

# 執行 RFE
rfe.fit(preprocessed_xtrain, ytrain)

In [ ]:
# 輸出被選入的特徵
select_index = rfe.get_feature_names_out()
# print(select_index)

# 想知道特徵名稱
select_index = [eval(i[1:]) for i in select_index]

# 特徵名稱
print( np.array(preprocessed_xtrain.columns)[select_index] )

['MSSubClass' 'LotFrontage' 'LotArea' 'OverallQual' 'OverallCond'
 'YearBuilt' 'YearRemodAdd' 'MasVnrArea' 'BsmtFinSF1' 'BsmtFinSF2'
 'BsmtUnfSF' 'TotalBsmtSF' '1stFlrSF' '2ndFlrSF' 'LowQualFinSF'
 'GrLivArea' 'BsmtFullBath' 'BsmtHalfBath' 'FullBath' 'HalfBath'
 'BedroomAbvGr' 'KitchenAbvGr' 'TotRmsAbvGrd' 'Fireplaces' 'GarageYrBlt'
 'GarageCars' 'GarageArea' 'WoodDeckSF' 'OpenPorchSF' 'EnclosedPorch'
 '3SsnPorch' 'ScreenPorch' 'PoolArea' 'MiscVal' 'MoSold' 'YrSold'
 'C (all)' 'FV' 'RH' 'RL' 'RM' 'Grvl' 'Pave' 'IR1' 'IR2' 'IR3' 'Reg' 'Bnk'
 'HLS' 'Low' 'Lvl' 'AllPub' 'NoSeWa' 'Corner' 'CulDSac' 'FR2' 'FR3'
 'Inside' 'Gtl' 'Mod' 'Sev' 'Blmngtn' 'Blueste' 'BrDale' 'BrkSide'
 'ClearCr' 'CollgCr' 'Crawfor' 'Edwards' 'Gilbert' 'IDOTRR' 'MeadowV'
 'Mitchel' 'NAmes' 'NPkVill' 'NWAmes' 'NoRidge' 'NridgHt' 'OldTown'
 'SWISU' 'Sawyer' 'SawyerW' 'Somerst' 'StoneBr' 'Timber' 'Artery' 'Norm'
 'Norm' 'Duplex' '1.5Fin' 'SLvl' 'CompShg' 'Tar&Grv' 'BrkComm' 'BrkFace'
 'CBlock' 'CemntBd' 'HdBoard' 'ImSt

# Recursive Feature Elimination with Cross-Validation

In [1]:
from sklearn.feature_selection import RFECV

In [2]:
model = XGBRegressor()

# 建立 RFECV 物件
rfecv = RFECV(estimator = model,
        min_features_to_select = 200,
        step = 5,
        cv = 5,
        scoring = "neg_mean_squared_error",
        verbose = 1)

# 執行 RFECV
rfecv.fit(preprocessed_xtrain, ytrain)

NameError: ignored

In [ ]:
rfecv.get_feature_names_out()

In [ ]:
rfecv.cv_results_